```mermaid
flowchart TB
A[广州数据]
B[上海数据]
C[深圳数据]
D[广州查询工具QueryEngineTool]
E[上海查询工具QueryEngineTool]
F[深圳查询工具QueryEngineTool]
G[文档查询选择工具get_tools]
H[生成prompt工具create_system_prompt]
I[创建代理工具create_agent]
J[最终代理builder_agent]

A--->D
B--->E
C--->F
D--->G
E--->G
F--->G
H--->J
G--->J
I--->J
```

### 1. 把QueryEngine转为工具使用，并使用tool_retriever.retrieve选择工具

在这个脚本中，首先构建了针对不同城市（广州、深圳、上海）的查询引擎，这些引擎被封装为工具（`QueryEngineTool`），并存储在 `tool_dict` 字典中。通过定义一个检索器 `tool_retriever` 并调用 `tool_retriever.retrieve(task)` 方法来选择与当前任务最相关的工具。这样可以确保在面对不同查询时，能够动态地从现有的工具集中选取合适的查询引擎进行信息检索。

### 2. 首先创建prompt，然后选择QueryEngine工具创建Agent

为了构建一个能够解决特定任务的代理（agent），首先需要生成一个系统提示（system prompt）。这一步通过调用 `create_system_prompt` 函数来完成。随后使用这个系统提示以及从 `tool_retriever.retrieve(task)` 选取的相关工具，利用 `ReActAgent.from_tools` 方法创建最终的代理。这种方法能够确保在每次面对新的查询任务时都能够动态生成和调整系统提示及工具集。

### 3. 嵌套Agent的使用

脚本中展示了如何构建一个多级代理（嵌套agent）。首先通过一系列步骤创建了一个顶层代理（builder_agent），它负责生成和管理其他具体的代理。这种设计允许在高层实现逻辑控制，同时利用具体执行层（如 `QueryEngineTool`）来处理细节任务。这样的结构可以提高系统的灵活性和可扩展性。

### 4. 嵌套的Agent带来流程不稳定，输出不如直接RAG靠谱

虽然多级代理（嵌套agent）提供了一种灵活的方式来构建复杂的任务解决系统，但在实际应用中可能会遇到一些挑战。例如，在某些情况下，顶层代理可能不能准确地判断出最适合当前任务的具体工具集或策略，这可能导致生成的系统提示不够精确或者查询结果偏离预期目标。此外，这种结构也可能增加代码复杂性和维护难度。

相比之下，直接使用RAG（Retrieval-Augmented Generation）方法在处理信息检索和回答问题时更为直接有效，因为它可以直接基于现有文档进行上下文依赖的信息提取与融合生成，通常能提供更稳定且高质量的输出结果。

In [1]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

base_url='http://localhost:11434'
llm = Ollama(model="qwen2.5:latest", request_timeout=360.0,base_url=base_url)
Settings.llm = llm
Settings.embed_model = OllamaEmbedding(model_name="quentinz/bge-large-zh-v1.5:latest",base_url=base_url)

## 获取数据

In [2]:
# 利用https://baike.deno.dev/获取百度百科查询地址
import re
import requests
from bs4 import BeautifulSoup

def get_html(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    
    # 检查响应状态码
    if response.status_code == 200:
        return response
    else:
        print(f"请求失败，状态码：{response.status_code}")
        return None

def build_md_from_html(html,save_path):
    soup = BeautifulSoup(html, 'html.parser')

    md_content=''
    # 获取summary部分
    md_content+='# 概览\n'
    summary_div= soup.find_all('div', class_='J-summary')  # 替换为实际的数据选择器
    summary_paragraphs = summary_div[0].find_all('div', {'data-tag': 'paragraph'})
    for paragraph in summary_paragraphs:
        text = paragraph.get_text(strip=True)
        text=re.sub(r'\[[1-9][0-9]*\]','',text)
        md_content+=text+'\n\n'
    
    # 获取内容部分
    content_div = soup.find_all('div', class_='J-lemma-content')  # 替换为实际的数据选择器
    paragraphs = content_div[0].find_all('div', {'data-tag': ['paragraph','header']})
    for paragraph in paragraphs:
        data_tag=paragraph['data-tag']
        text = paragraph.get_text(strip=True)

        if data_tag=='header':
            text=text.replace('播报编辑','')
            data_level=int(paragraph['data-level'])
            text='#'*data_level+' '+text+'\n'
        else:
            text=re.sub(r'\[[1-9][0-9]*\]','',text)
            text+='\n\n'
        
        md_content+=text

    # 将结果写入文件
    with open(save_path, 'w', encoding='utf-8') as file:
        file.write(md_content)

cities = ["广州", "深圳", "上海"]
# for city in cities:
#     url = f"https://baike.deno.dev/item/{city}"
#     response = get_html(url).json()
#     baike_url=response['data']['link']
#     print(f"{city} 的链接是：{baike_url}")

#     html=get_html(baike_url).text
#     print(f'get {city} html done')

#     save_path=f'../../data/citys/{city}.md'
#     build_md_from_html(html,save_path)
#     print(f'save {city} markdown')


## 每个文件构建查询引擎

In [3]:
from llama_index.core import SimpleDirectoryReader

city_docs = {}
for city in cities:
    city_docs[city] = SimpleDirectoryReader(
        input_files=[f"../../data/citys/{city}.md"]
    ).load_data()

In [4]:
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool,ToolMetadata

tool_dict = {}

for city in cities:
    index=VectorStoreIndex.from_documents(documents=city_docs[city])

    query_engine=index.as_query_engine()

    tool=QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=city,
            description=(f"关于{city}问题的回答助手")))
    tool_dict[city]=tool

## prompt->选择查询工具->创建Agent进行增强生成

In [6]:
from llama_index.core.objects import ObjectIndex

tool_index=ObjectIndex.from_objects(
    list(tool_dict.values()),
    index_cls=VectorStoreIndex
)

tool_retriever=tool_index.as_retriever(similarity_top_k=1)

In [7]:
from llama_index.core.llms import ChatMessage
from llama_index.core import ChatPromptTemplate
from llama_index.core.agent import ReActAgent
from typing import List
from llama_index.core.agent import FunctionCallingAgent

GEN_SYS_PROMPT_STR = """\
Task information is given below. 

Given the task, please generate a system prompt for an  bot to solve this task: 
{task} \
"""

gen_sys_prompt_messages = [
    ChatMessage(
        role="system",
        content="You are helping to build a system prompt for another bot.",
    ),
    ChatMessage(role="user", content=GEN_SYS_PROMPT_STR),
]
GEN_SYS_PROMPT_TMPL = ChatPromptTemplate(gen_sys_prompt_messages)


agent_cache = {}

def create_system_prompt(task: str)-> str:
    """Create system prompt for another agent given an input task."""
    fmt_messages = GEN_SYS_PROMPT_TMPL.format_messages(task=task)
    response = llm.chat(fmt_messages)
    return response.message.content


def get_tools(task: str) -> List[str]:
    """Get the set of relevant tools to use given an input task."""
    subset_tools = tool_retriever.retrieve(task)
    tool_names= [t.metadata.name for t in subset_tools]
    return tool_names


def create_agent(system_prompt: str, tool_names: List[str]) -> str:
    ''' Create an agent given a system prompt and an input set of tools
    system_prompt :  prompt created by create_system_prompt
    tool_names : tools can only choice 广州 上海 or 深圳
    '''
    try:
        # print('tool_names',tool_names)
        # get the list of tools
        input_tools = [tool_dict[tn] for tn in tool_names]
 
        agent = ReActAgent.from_tools(input_tools, verbose=True)
        agent_cache["agent"] = agent
        return_msg = "Agent created successfully."
    except Exception as e:
        return_msg = f"An error occurred when building an agent. Here is the error: {repr(e)}"
    return return_msg

In [8]:
from llama_index.core.tools import FunctionTool

system_prompt_tool = FunctionTool.from_defaults(fn=create_system_prompt)
get_tools_tool = FunctionTool.from_defaults(fn=get_tools)
create_agent_tool = FunctionTool.from_defaults(fn=create_agent)

## 创建汇总Agent

In [9]:
GPT_BUILDER_SYS_STR = """\
You are helping to construct an agent given a user-specified task. 
You should generally use the tools in this order to build the agent.

1) Create system prompt tool: to create the system prompt for the agent.
2) Get tools tool: to fetch the candidate set of tools to use.
3) Create agent tool: to create the final agent.
"""

prefix_msgs = [ChatMessage(role="system", content=GPT_BUILDER_SYS_STR)]


builder_agent = ReActAgent.from_tools(
    tools=[system_prompt_tool, get_tools_tool, create_agent_tool],
    prefix_messages=prefix_msgs,
    verbose=True,
    max_iterations=20
)

In [10]:
response=builder_agent.query("创建一个代理，告诉我广州所有的4A级旅游景区？")

> Running step 7294cd7f-761d-4370-bded-0458eab19207. Step input: 创建一个代理，告诉我广州所有的4A级旅游景区？
Thought: 我需要使用工具来获取广州的所有4A级旅游景区的信息。
Action: get_tools
Action Input: {'task': '查询广州所有4A级旅游景区'}
Observation: ['广州']
> Running step 7213e094-69db-43a0-a8dc-af1990504265. Step input: None
Thought: 根据返回的结果，我将创建一个代理以获取广州的4A级旅游景区信息。
Action: create_agent
Action Input: {'system_prompt': '请提供广州所有的4A级旅游景区的名字。', 'tool_names': ['广州']}
Observation: Agent created successfully.
> Running step b34079a0-a39f-4215-a290-77e6e5202ce1. Step input: None
Thought: 我可以回答这个问题了，由于代理已经创建成功，将使用它来获取广州的4A级旅游景区信息。
Action: create_system_prompt
Action Input: {'task': '查询广州所有4A级旅游景区'}
Observation: 当然，以下是一个系统提示，用于指导助手完成这个任务：

```plaintext
您需要帮助查询广州市的所有4A级旅游景区。请按照以下步骤操作：

1. 使用可靠的旅游网站、政府官方网站或相关的数据库来查找广州市的所有4A级旅游景区。
2. 收集每个景区的详细信息，包括名称、地址、开放时间、门票价格等基本信息。
3. 将这些信息整理成一个列表或者表格形式，并尽可能提供最新的数据以确保信息准确无误。
4. 根据用户的查询需求，可以进一步提供一些热门或特色景点的具体介绍。

请确保提供的信息是最新且准确的。如果有任何不确定的信息，请不要做出判断，而是找到最可靠的数据源进行确认后再提供给用户。
```

这样设置可以帮助助手更好地理解任务要求并给出详尽的回答。
> Running 

In [13]:
response=builder_agent.query("告诉我广州和上海的商业街？")

> Running step 6f0a65ea-0baf-4229-b506-c33e03697fd4. Step input: 告诉我广州和上海的商业街？
Thought: 我需要使用工具来获取广州和上海的商业街信息。
Action: get_tools
Action Input: {'task': '提供广州和上海的商业街信息'}
Observation: ['上海']
> Running step 854983d9-c39d-4a36-b34f-2475821a2182. Step input: None
Thought: 工具返回的结果只有上海的信息，没有广州的信息。我将使用该工具获取广州的商业街信息。
Action: get_tools
Action Input: {'task': '提供广州的商业街信息'}
Observation: ['广州']
> Running step 3e1bc7a6-9fea-4371-873c-69972fe6f26e. Step input: None
Thought: 现在我得到了广州和上海的一些商业街信息，但需要更具体的街道名称。我将使用这些工具创建代理来获取更多信息。
Action: create_agent
Action Input: {'system_prompt': '请提供广州和上海著名商业街的详细信息，包括街道名称、特色商铺等。', 'tool_names': ['广州', '上海']}
Observation: Agent created successfully.
> Running step 089c430a-0a88-4e86-b056-7f40c74a4980. Step input: None
Thought: 我已经成功创建了一个代理来获取广州和上海著名商业街的信息。我现在可以回答用户的问题了。
Answer: 广州的著名商业街有天河路步行街、北京路步行街等，而上海则以南京东路步行街、淮海中路等著称。这些地方不仅拥有各种各样的特色商铺，同时也是体验当地文化和消费的好去处。
